# Stableswap Shoot-out
> "Is Curve the King of the Stablecoin Dex Market?"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Curve]

In [2]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


     |████████████████████████████████| 23.9 MB 14 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


dplyr.summarise.inform,[RTYPES.NILSXP]


In [8]:
#hide
%%R
#Grab base query from Flipside
df_curve = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/7bc1fde9-2fb5-402b-86af-e3ec4ca88f73/data/latest', simplifyDataFrame = TRUE)
df_balancer = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/910644d0-7ffe-405f-956d-dec64aabf113/data/latest', simplifyDataFrame = TRUE)
df_univ3 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/f6954c9c-dcee-4337-8b03-06828eeda38a/data/latest', simplifyDataFrame = TRUE)
df_univ2 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/59b40506-899d-421a-8897-71a6b3d02b12/data/latest', simplifyDataFrame = TRUE)
#df_sushi = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/955a8c81-d2d2-48ac-b34a-ba28ae3cf2c0/data/latest', simplifyDataFrame = TRUE)
df_saddle = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/fea29310-8feb-408e-bd04-21022e6dc6b9/data/latest', simplifyDataFrame = TRUE)

#Bind 'em together
df <- df_curve %>% mutate(dex = "Curve") %>%
  bind_rows(df_balancer %>% mutate(dex = "Balancer")) %>%
  bind_rows(df_univ3 %>% mutate(dex = "Uniswap v3")) %>%
  bind_rows(df_univ2 %>% mutate(dex = "Uniswap v2")) %>%
 # bind_rows(df_sushi %>% mutate(dex = "sushi")) %>%
  bind_rows(df_saddle %>% mutate(dex = "Saddle")) 

#fix the column names
names(df)<-tolower(names(df))

#Change the date to date format
df$block_ts <- parse_datetime(df$block_ts)

#add swap size groups
df <- df %>% mutate(swap_size_bucket = case_when(
                                           #source_amount < 10 ~ '10',
                                           #source_amount < 100 ~ '100',
                                           source_amount < 1000 ~ '<1k',
                                           source_amount < 10000 ~ '<10k',
                                           source_amount < 100000 ~ '<100k',
                                           source_amount < 1000000 ~ '<1m',
                                           source_amount < 10000000 ~ '<10m',
                                           #source_amount < 100000000 ~ '100m',
                                           TRUE ~ '>100m'
                                           ))
df$swap_size_bucket <- parse_factor(df$swap_size_bucket, 
                                    ordered = TRUE, 
                                    levels = c('<10','<100','<1k','<10k','<100k','<1m','<10m','<100m','<1b','>100m'))
#add swap pair
df <- df %>% mutate(swap_pair = paste(source_symbol, dest_symbol, sep = "-"))
df$swap_pair <- parse_factor(df$swap_pair,
                             levels = c('USDC-DAI', 'DAI-USDC', 'USDC-USDT', 'USDT-USDC', 'DAI-USDT', 'USDT-DAI'))

#Roll up over all data by dex
df_by_dex <- df %>% group_by(dex) %>%
  summarise(exchange_no_gas = sum(dest_amount) / sum(source_amount),
            exchange_net = sum(dest_amount) / (sum(source_amount) + sum(tx_fee_usd)),
            volume = sum(source_amount),
            cost_no_gas = (1 - exchange_no_gas)*100,
            cost_net = (1-exchange_net)*100) %>%
  arrange(desc(exchange_net))

#Roll up by dex & swap size bucket
df_by_swap_size <- df %>% group_by(dex, swap_size_bucket) %>%
  summarise(exchange_no_gas = sum(dest_amount) / sum(source_amount),
            exchange_net = sum(dest_amount) / (sum(source_amount) + sum(tx_fee_usd)),
            volume = sum(source_amount),
            cost_no_gas = (1 - exchange_no_gas)*100,
            cost_net = (1-exchange_net)*100) %>%
  arrange(swap_size_bucket, desc(exchange_net))

#Roll up by dex & pair
df_by_swap_pair <- df %>% group_by(dex, swap_pair) %>%
  summarise(exchange_no_gas = sum(dest_amount) / sum(source_amount),
            exchange_net = sum(dest_amount) / (sum(source_amount) + sum(tx_fee_usd)),
            volume = sum(source_amount),
            cost_no_gas = (1 - exchange_no_gas)*100,
            cost_net = (1-exchange_net)*100) %>%
  arrange(swap_pair, desc(exchange_net))

# Stablecoin Swaps

Stablecoins arguably form the basis for decentralised finance on Ethereum.  Whether it's borrowing, lending, liquidity provision or yield farming, there are almost always stablecoins in the middle of it.  The stablecoins we are referring to here are the 3 big USD pegged stablecoins -  USDC, USDT and DAI.  Defi is full of decentralised token exchanges - dex - which exchange one token for another.  A specialised use case of the dex is stablecoin swaps - exchanging between USDC, USDT and DAI either to get the right token for the job users need to do, or to arbitrage between 3 assets that should be worth USD 1 each but often are not exactly.  As the value of each of these coins is close to equal, a number of optimisations to the traditional AMM dex model have evolved for stablecoin swaps.  The intent of this post is to look at the efficiency of these alternatives & see which one gives the best results for users.  

## Stableswap Efficiency
Firstly we should define what we mean by efficiency of a stablecoin swap.  Let's imagine a magical world where USDC, USDC and DAI are worth exacly 1 USD each.  Right now, it's really close to that, but certain parties assign different risk discounts to each coin and it is not always the case.  We will make the assumption that these coins are equivalent in underlying value as the arbitrage trade is enormous and relatively efficient.  A perfect stablecoins swap should therefore be receiving tokens at a 1:1 ratio.  It is almost always less than this, due to four areas of loss for the user:
- Swap fees - a dex will generate revenue by taking a percentage of each swap.  It is a competitive market for stableswaps, and these fees are often very low - 0.05% for Uniswap v3 and 0.03% for Curve on stablecoin swaps.
- Price Impact - The quoted price on a dex is the price for the next token swapped, regardless of the size of the trade.  Dex type exchanges rely on a pool of liquidity for each token in the swap, with the actual swap price being a function of the ratio of the tokens in the pool *after* the swap occurs.  If the swap is a significant portion of the pool liquidity it can move the realised price that the user receives.  This can be mitigated by the design of the AMM, particularly with stablecoin swaps where the swap range between tokens is very narrow.  This is a key component of our analysis here - dex protocols don't just compete on fees, they also compete on the swap curve design.  A number of our target protocols have optimised the swap curve to minimise price impact on swaps.
- Slippage - a blockchain is a dynamic environment, being used by potentially millions of users at any one time.  Slippage occurs when the quoted price of a swap changes due to the action of other simultaneous market participants.  A user can submit a swap at a given price, but if a large transaction occurs at around the same time, the user may receive more or less than expected because of this transaction.  This is known as slippage, and most dex UIs include a limit on slippage tolerance for the user.  If the slippage is more than the tolerance, the transaction will fail.
- Blockchain Transaction Fees - the Ethereum network uses the ETH token to pay for blockspace in the blockchain, and this payment is called gas.  Blockspace is a limited resource and there is a free market for inclusion.  Prices are high during times of congestion and lower at other times.  Gas prices form a significant portion of dex impact for users, particularly with lower value swaps.

The focus of this post is the user - what does the user get in a stablecoins swap, compared with the 1:1 ideal situation?

## Dex Models
Before we begin, an outline of each dex model is in order:

#### Uniswap V2 - Constant Product AMM
Uniswap launched as the OG Automated Market Maker dex, based on a [Reddit post by Vitalik in 2016](https://www.reddit.com/r/ethereum/comments/55m04x/lets_run_onchain_decentralized_exchanges_the_way/).  It uses the constant product model - x * y = k, where the price of a token in a dex pool is equal to the ratio of that token to the other token in the pool.  Uniswap was hugely successful with this model, spawning many imitators. 

#### Curve - Optimised Swap Curves for Stableswaps
Constant product dex markets are subject to price impact and slippage.  They are optimised for swapping between a large number of tokens, rather than efficient swaps between a small number.  Michael Egorov, the founder of Curve, recognised this and founded Curve, with a swap curve more efficient than the constant product model for swaps of tokens of a similar value.  It was enormously successful and has attracted over USD 12b of liquidity.  Curve has the [Tripool](https://curve.fi/3pool/) - an AMM pool for USDC, USDT and DAI with over USD 2b of liquidity alone.

#### Uniswap V3 - Targeted Liquidity
In the constant product AMM model, the liquidity is spread over all prices for both tokens.  For pegged stablecoins, where the price of each coin is well within 1% of the price of the other coins, this means that much of the liquidity is never utilised.  Uniswap launched V3 of their protocol, allowing users to target their liquidity around specified price points.  This had a major impact on the capital efficiency of stablecoin swap pools and helped to close the market gap to Curve.

#### Balancer
Balancer uses a single vault model, where users interact with a single contract for all swaps.  The swap logic is abstracted away to underlying pools so that the user only pays gas fees on the vault transactions.  The underlying pools implement optimised swap curves for the target tokens, giving users advantages in swap efficiency and gas usage.

#### Saddle
Imitation is the sincerest form of flattery.  Saddle launched to outcries of copyright infringement with their Curve inspired stableswap AMM.  Let's see how it stacks up.


# Judging the Stableswap Options

To analyse which is the best option for users when swapping stablecoins, we collected on-chain data from [Flipside Crypto](https://flipsidecrypto.com).  This data looked at swaps between the three major pegged stablecoins (USDC, DAI, USDT) on the five platforms described above (Curve, Uniswap V2 & V3, Balancer, Saddle).  Sushi was excluded from the analysis as it carries very low liquidity in stable-stable pair pools - it appears most stableswaps are done via intermediate tokens like ETH.  Data was sourced which met the following criteria:
- Swaps occured during the last 2 months (to 2021-09-17)
- Swaps weren't done via aggregators like 1inch or Paraswap - direct with the protocol only
- Swaps weren't routed between intermediate pairs - they were direct in & out of the stablecoin pools
- Transactions with zero gas fees were excluded - services like Flashbots allow advanced users to substitute gas costs with direct payments to block miners.

The analysis centered on the return to the users - for every token swapped in a stablecoin swap, how much was lost in the transaction.  Assuming that each of our stablecoins is equivalent in value (1 USD), this was calculated by dividing the received token amount by the submitted token amount.  This was done in two ways:
- Excluding gas costs - looking at swap fees, price impact & slippage only.  This give insight into the underlying design & liquidity of the swap protocol.
- Including gas costs - the net outcome for the user.  *The only thing that really matters.*

*Note on Balancer - Balancer swaps can include a rebate in BAL tokens for user gas costs.  These have not been considered in this analysis, because they have been interpreted as a usage incentive which may not persist, rather than as an underlying part of the protocol.*

# Overall Performance Across all Stableswap Pairs

The first analysis makes the assumption that USDC, DAI and USDC are equivalent in value, so we don't differentiate between swap pairs (yet).  For all the transactions which meet the criteria in the paragraph above, we look at the transaction cost to the user.  This transaction cost, expressed as a percentage, is the amount lost compared to the ideal 1:1 swap from a perfect exchange.  More formally, it is:

> $TransactionCost\% = (1 - \frac{AmountIn}{AmountOut}) \times 100 $

For example, if the user puts in 100 USDC and receives 99 DAI, then the transaction cost % would be 1%.  Lower is better.

## Excluding Gas

The chart below shows the transaction cost % across all pairs over the last 2 months, excluding the impacts of gas.  As we can see, Curve is the most cost efficient dex, with an average transaction cost impact of under .04%.  Curve charges 0.03% as a protocol fee on the Tripool, so there is around 0.01% of slippage & price impact on stablecoin trades on Curve.  Compare this to Uniswap v3, which has 0.05% fees.  The Uniswap v3 overall cost impact is *less* than the fees, indicating that there may be some positive slippage & price impact for users on the platform.  Here we also see the benefits of optimising for stablecoin swaps over a generall AMM model.  The Uniswap V2 platform - built for swapping many different tokens over all price ranges - is an order of magnitude more costly than the optimised stableswap dex environments.



In [3]:
#hide_input
# Plot the top 10
df_p = %R df_by_dex %>% arrange(cost_no_gas) 
fig = px.bar(df_p
             , x = "dex"
             , y = "cost_no_gas"
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , labels=dict(dex="Protocol", cost_no_gas="Transaction Cost %")
             , title= "Total Transaction Costs Excluding Gas"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Transaction Cost %')
fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
fig.show()

# Including Gas

It's no secret that gas costs have been prohibitive on Ethereum over the past 2 months.  The NFT boom has driven up competition for blockspace and gas levels have been consistently high, as well as suffering from extraordinary short term spikes.  When we factor this into the analysis the results are as below.  Curve is still the most efficient dex overall, with an average impact of 0.045%.  Uniswap v3 is next, then there is daylight to the others.  As gas costs in a transaction are a fixed amount the impact of gas will depend on the swap size.  There is probably a swap size influence on these numbers below - this will be investigated further.

In [4]:
#hide_input
# Plot the top 10
df_p = %R df_by_dex %>% arrange(cost_net) 
fig = px.bar(df_p
             , x = "dex"
             , y = "cost_net"
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , labels=dict(dex="Protocol", cost_net="Transaction Cost %")
             , title= "Total Transaction Costs Including Gas"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Transaction Cost %')
fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
fig.show()

# Performance by Swap Size

The charts below show how much volume is swapped on the exchanges in different swap size classes.  We can see that for swap sizes below USD 100k, Uniswap has the most volume across both v2 and v3.  For swap sizes between 100k and 1m, Curve and Uniswap v3 dominate.  Above 1m, Curve is the clear choice for stablecoin swaps.  We can conclude that for small swaps, users favour the popularity & ease of use of the Uniswap platform, but for serious sized swaps there is a greater sophistication and a desire to get the best deal possible.  We also see that Balancer & Saddle have insignificant swap amounts compared with the others.  It's worth reinforcing that the swaps investigated here are only those done directly via the protocols - much of the volume routed through Balancer & Saddle are likely coming from aggregators and other protocols. 

In [7]:
#hide_input
# Plot the top 10
df_p = %R df_by_swap_size %>% arrange(swap_size_bucket, desc(volume)) 
fig = px.bar(df_p
             , x = "dex"
             , y = "volume"
             , facet_col = 'swap_size_bucket'
             , facet_col_wrap=3
             , facet_col_spacing=0.03
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , category_orders={
                 "dex": ["Curve", "Uniswap v3", "Balancer", "Uniswap v2", "Saddle"]
                 }
             , labels=dict(dex="Protocol", cost_no_gas="Transaction Cost %", volume="Volume USD", swap_size_bucket="Swap Size")
             , title= "Transaction Volume by Swap Size"
             , template="simple_white", width=1000, height=1000/1.618
             )
#fig.update_yaxes(title_text='Transaction Cost %')
#fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)
fig.show()

# Excluding Gas Costs

The graph below shows the transaction costs, excluding gas, segmented by swap size.  The charts are relatively consistent, with Curve being the best performer across all classes.  An interesting trend is the increase in costs in Uniswap v2 as the swap size increases.  This is due to price impacts in the pool - the larger swaps are a big enough proportion of the total pool liquidity to impact the price.  

In [9]:
#hide_input
# Plot the top 10
df_p = %R df_by_swap_size %>% arrange(swap_size_bucket, cost_no_gas) 
fig = px.bar(df_p
             , x = "dex"
             , y = "cost_no_gas"
             , facet_col = 'swap_size_bucket'
             , facet_col_wrap=3
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , labels=dict(dex="Protocol", cost_no_gas="Transaction Cost %", swap_size_bucket="Swap Size")
             , title= "Total Transaction Costs Excluding Gas"
             , template="simple_white", width=1000, height=1000/1.618
             )
#fig.update_yaxes(title_text='Transaction Cost %')
#fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
fig.show()

## Including Gas Costs

Now we see the real outcome for the user, with gas costs included.  The charts below show the total transaction cost, including gas, across the different swap size cohorts.  Please note the scales are now different on each chart to allow better comparison.  We see that for smaller swap sizes (< 10k), the gas efficiency of Uniswap v3 makes this the cheapest option for users.  There is a significant margin in this advantage for very small swaps (< 1k).  As swap sizes increase, however, the underlying protocol and liquidity advantage of Curve washes away the gas costs and we see Curve as the best performing dex for swap sizes larger than USD 10k

In [9]:
#@title
#hide_input
# Plot the top 10
df_p = %R df_by_swap_size %>% arrange(swap_size_bucket) 
fig = px.bar(df_p
             , x = "dex"
             , y = "cost_net"
             , facet_col = 'swap_size_bucket'
             , facet_col_wrap=3
             , facet_col_spacing=0.03
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , category_orders={
                 "dex": ["Curve", "Uniswap v3", "Balancer", "Uniswap v2", "Saddle"]
                 }
             , labels=dict(dex="Protocol", cost_net="Transaction Cost %", swap_size_bucket="Swap Size")
             , title= "Total Transaction Costs Including Gas"
             , template="simple_white", width=1000, height=1000/1.618
             )
#fig.update_yaxes(title_text='Transaction Cost %')
#fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=True)
#fig.update_xaxes(matches=None)
fig.show()

# Return by Swap Pair

To confirm the conclusions above, we will look to see if there is any influence on transaction costs based on the swap pair.  The chart below shows the volume of swaps by swap pair.  There is a difference in volume between the most popular and least popular swap directions of around 3x - we will see if this influences the transaction cost outcomes.

In [11]:
#hide_input
df_p = %R df %>% group_by(swap_pair) %>% summarise(volume = sum(source_amount)) %>% arrange(swap_pair)
fig = px.bar(df_p
             , x = "swap_pair"
             , y = "volume"
             , labels=dict(swap_pair="Swap Pair", volume="Volume USD")
             , title= "Transaction Volume USD"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Volume USD')
fig.update_xaxes(title_text='Swap Pair')
fig.update_layout(showlegend=False)
fig.show()

## Transaction Cost by Swap Pair

The charts below shows the transaction costs, including gas, split by the swap pairs.  The trend is relatively consistent across all pairs, and conistent with the overall conclusions we have made. Curve is the best performing dex across the board, followed by Uniswap v3.  An interesting feature of these charts are the DAI-USDT and DAI-USDC performance of Curve - the transaction costs are much lower than the average, probably due to favourable price impact in the swaps.  For the DAI-USDC swap direction there is a negative cost for the user - the user receives more USDC than the DAI swapped in.

In [14]:
#hide_input
# Plot the top 10
df_p = %R df_by_swap_pair %>% arrange(swap_pair) 
fig = px.bar(df_p
             , x = "dex"
             , y = "cost_net"
             , facet_col = 'swap_pair'
             , facet_col_wrap=3
#             , facet_col_spacing=0.03
             , color = "dex"
             , color_discrete_map = {
                 'Curve': '#FF1A00',
                 'Balancer': '#1F77B4',
                 'Saddle': '#1F77B4',
                 'Uniswap v3': '#1F77B4',
                 'Uniswap v2': '#1F77B4',
             }
             , category_orders={
                 "dex": ["Curve", "Uniswap v3", "Balancer", "Uniswap v2", "Saddle"]
                 }
             , labels=dict(dex="Protocol", cost_net="Transaction Cost %", swap_pair="Swap Pair")
             , title= "Total Transaction Costs Including Gas"
             , template="simple_white", width=1000, height=1000/1.618
             )
#fig.update_yaxes(title_text='Transaction Cost %')
#fig.update_xaxes(title_text='Protocol')
fig.update_layout(showlegend=False)
#fig.update_yaxes(matches=None)
#fig.update_yaxes(showticklabels=True)
#fig.update_xaxes(matches=None)
fig.show()

# Conclusions

In attempting to do a fair comparison between stablecoin swap platforms, we examined nearly $3b of transactions over a 2 month period, looking at swaps done directly with these swap platforms.  We found the following:
- Curve is, on aggregate, the cheapest place to swap the big 3 stablecoins.
- For smaller transactions (< 10k USD), Uniswap v3 is the cheaper platform due to gas cost efficiency
- For larger transactions than 10k, Curve is the clear winner
- There is a big advantage in transaction costs using a specialised stablecoin swap platform, as opposed a general purpose dex like Uniswap v2
- Simply copying an existing concept is not a recipe for instant success for Saddle - the weight of liquidity in Curve translates to better outcomes for users, which begets even more liquidity

If you are a user looking to swap stablecoins (USDC, USDT, DAI), then Curve should be the first choice.  It's worth a comparison with Uniswap v3, however, as it will be slightly more gas efficient on smaller trades.  Happy swapping.

# References:
- All data was sourced from the curated on-chain data tables at [Flipside Crypto](https://flipsidecrypto.com)  
- I learned a lot about dex pricing impacts from this article by [Hasu](https://twitter.com/hasufl) at https://research.paradigm.xyz/amm-price-impact  
- This post was inspired by this article at the fantastic Curve Market Cap
https://curve.substack.com/p/august-18-2021-curve-vs-balancer


In [7]:
#%R df_by_swap_size %>% arrange(swap_size_bucket, dex) 
#%R df_by_swap_size %>% arrange(swap_size_bucket, cost_no_gas) 
import plotly
plotly.__version__

'5.3.1'